# Hospital Patient Data Analysis

1.	Load the patient dataset and show summary with info().

In [2]:
import pandas as pd  # Importing pandas library for data manipulation and analysis

# Reading patient records dataset into a DataFrame
patients_data_df = pd.read_csv('patient_data.csv')

# Reading billing records dataset into a DataFrame
billing_data_df = pd.read_csv('billing_data.csv')

In [3]:
patients_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [4]:
patients_data_df.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [5]:
patients_data_df.describe(include='all')

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
count,6.000000,6,6,6,4.000000,6.000000,6
unique,NaN,5,4,4,NaN,NaN,5
top,NaN,Alice,Cardiology,Dr. Smith,NaN,NaN,2023-01-10 09:00
freq,NaN,2,3,3,NaN,NaN,2
mean,102.666667,NaN,NaN,NaN,5925.000000,1.666667,NaN
std,1.632993,NaN,NaN,NaN,1192.686044,0.816497,NaN
min,101.000000,NaN,NaN,NaN,5000.000000,1.000000,NaN
25%,101.250000,NaN,NaN,NaN,5000.000000,1.000000,NaN
50%,102.500000,NaN,NaN,NaN,5600.000000,1.500000,NaN
75%,103.750000,NaN,NaN,NaN,6525.000000,2.000000,NaN


2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].

In [6]:
# Create a new DataFrame containing only the selected columns for billing information

cols_keep = ['PatientID','Department','Doctor','Billamount']
patients_billing = patients_data_df.copy([cols_keep])

3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].

In [7]:
# Dropping admin-related columns to keep only useful patient information
admin_cols = ['ReceptionistID','CheckInTime']
patients_data_df.drop(columns=admin_cols, errors= 'ignore')

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN
5,101,Alice,Cardiology,Dr. Smith,5000.0


4.	Use groupby to find total bill amount per department.

In [8]:
# calculate the sum of 'BillAmount' for each department.
dept_totals=patients_billing.groupby('Department', as_index=False)['BillAmount'].sum()

dept_totals.sort_values('BillAmount',ascending=False)   # and sort the departments by total billing amount in descending order
print(dept_totals)

    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


5.	Remove duplicate patient records based on PatientID.

In [9]:
# Aggregate patient billing data by PatientID:
agg_rules = {
    'Department': 'last',
    'Doctor': 'last',
    'BillAmount': 'sum'
}                             # - Keep the last recorded Department and Doctor for each patient

patients_agg = patients_billing.groupby('PatientID',as_index=False).agg(agg_rules)
print(patients_agg)

   PatientID   Department     Doctor  BillAmount
0        101   Cardiology  Dr. Smith     10000.0
1        102    Neurology   Dr. John         0.0
2        103  Orthopedics    Dr. Lee      7500.0
3        104   Cardiology  Dr. Smith      6200.0
4        105  Dermatology   Dr. Rose         0.0


6.	Fill missing BillAmount values with the mean bill amount.

In [23]:
mean_bill = patients_billing['BillAmount'].mean(skipna=True)
      # Calculate the mean of 'BillAmount' (ignoring NaN) and replace all missing values with this mean
patients_billing['BillAmount']= patients_billing['BillAmount'].fillna(mean_bill)


print("Mean Bill:", mean_bill)
print("\nupdated DataFrame:")
print(patients_billing.head())

Mean Bill: 5925.0

updated DataFrame:
   PatientID     Name   Department     Doctor  BillAmount  ReceptionistID  \
0        101    Alice   Cardiology  Dr. Smith      5000.0               1   
1        102      Bob    Neurology   Dr. John      5925.0               2   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0               1   
3        104    David   Cardiology  Dr. Smith      6200.0               3   
4        105      Eva  Dermatology   Dr. Rose      5925.0               2   

        CheckInTime  
0  2023-01-10 09:00  
1  2023-01-11 10:30  
2  2023-01-12 11:00  
3  2023-01-13 12:00  
4  2023-01-14 08:45  


7.	Merge the billing dataset with patient dataset on PatientID.

In [11]:
# Convert the 'InsuranceCovered' column to numeric, coercing any non-numeric values to NaN.
billing_data_df['InsuranceCovered']=pd.to_numeric(billing_data_df['InsuranceCovered'],errors='coerce')

merged= pd.merge(patients_agg, billing_data_df, on='PatientID',  how='left', suffixes=('_patients','_billing'),validate='m:1')
merged.info()# Then merge the aggregated patient data (patients_agg) with billing data (billing_data_df) 
# 'validate="m:1"' ensures that each patient in billing_data_df corresponds to at most one patient in patients_agg.

print(merged.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PatientID         5 non-null      int64  
 1   Department        5 non-null      object 
 2   Doctor            5 non-null      object 
 3   BillAmount        5 non-null      float64
 4   InsuranceCovered  5 non-null      int64  
 5   FinalAmount       5 non-null      int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 372.0+ bytes
   PatientID   Department     Doctor  BillAmount  InsuranceCovered  \
0        101   Cardiology  Dr. Smith     10000.0              2000   
1        102    Neurology   Dr. John         0.0              1500   
2        103  Orthopedics    Dr. Lee      7500.0              2500   
3        104   Cardiology  Dr. Smith      6200.0              3000   
4        105  Dermatology   Dr. Rose         0.0              1000   

   FinalAmount  
0         3000  
1 

8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).

In [22]:
# Adding new patient records to the existing patients_data_df DataFrame by creating a new DataFrame (new_patients) 
new_patients = pd.DataFrame([
    {'PatientID': '106', 'name': 'Ray','Department': 'Cardiology', 'Doctor': 'Dr. Smith', 'BillAmount': 5000.0, 'ReciptionistID': 1,'CheckInTime': '2023-10-15 11:00' },
    {'PatientID': '107', 'name': 'Jason','Department': 'Neurology', 'Doctor': 'Dr. John', 'BillAmount': 6500.0, 'ReciptionistID': 3,'CheckInTime': '2023-10-11 10:00' },

])

patients_data_df= pd .concat([patients_data_df, new_patients], ignore_index=True, sort=False) # and concatenating it. ignore_index=True ensures the index is reset, and sort=False keeps columns in original order.


print("\nUpdated DataFrame:")
print(patients_data_df)


Updated DataFrame:
   PatientID     Name   Department     Doctor  BillAmount  ReceptionistID  \
0        101    Alice   Cardiology  Dr. Smith      5000.0             1.0   
1        102      Bob    Neurology   Dr. John         NaN             2.0   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0             1.0   
3        104    David   Cardiology  Dr. Smith      6200.0             3.0   
4        105      Eva  Dermatology   Dr. Rose         NaN             2.0   
5        101    Alice   Cardiology  Dr. Smith      5000.0             1.0   
6        106      NaN   Cardiology  Dr. Smith      5000.0             NaN   
7        107      NaN    Neurology   Dr. John      6500.0             NaN   
8        106      NaN   Cardiology  Dr. Smith      5000.0             NaN   
9        107      NaN    Neurology   Dr. John      6500.0             NaN   
10       106      NaN   Cardiology  Dr. Smith      5000.0             NaN   
11       107      NaN    Neurology   Dr. John      6500.

9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

In [24]:
# Add insurance info and calculate final bill, then merge with new_patients DataFrame.

new_cols = pd.DataFrame({
    'InsuranceCovered': [True, False, True, False][: len(new_patients)],
    'FinalAmount':
    new_patients['BillAmount'] * 0.8
})

final_df = pd.concat([new_patients.reset_index(drop=True),new_cols], axis=1)


print("Final DataFrame:\n")
print(final_df)

Final DataFrame:

  PatientID   name  Department     Doctor  BillAmount  ReciptionistID  \
0       106    Ray  Cardiology  Dr. Smith      5000.0               1   
1       107  Jason   Neurology   Dr. John      6500.0               3   

        CheckInTime  InsuranceCovered  FinalAmount  
0  2023-10-15 11:00              True       4000.0  
1  2023-10-11 10:00             False       5200.0  


Expected Outcome:

•	Final cleaned dataset with accurate billing info.

In [21]:
# Display the structure and summary of the final dataset, including column names, data types, and non-null counts

print('\nFinal dataset info:')
print(final_df.info())


Final dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PatientID         2 non-null      object 
 1   name              2 non-null      object 
 2   Department        2 non-null      object 
 3   Doctor            2 non-null      object 
 4   BillAmount        2 non-null      float64
 5   ReciptionistID    2 non-null      int64  
 6   CheckInTime       2 non-null      object 
 7   InsuranceCovered  2 non-null      bool   
 8   FinalAmount       2 non-null      float64
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 262.0+ bytes
None


•	All missing values handled, merged dataset across PatientID.

In [15]:
# Check and display the number of missing (NaN) values in each column of the DataFrame

print('\nMissing values per column:')
print(final_df.isna().sum())


Missing values per column:
PatientID           0
name                0
Department          0
Doctor              0
BillAmount          0
ReciptionistID      0
CheckInTime         0
InsuranceCovered    0
FinalAmount         0
dtype: int64


•	Ability to perform further analytics on department-wise revenue or doctor performance.

In [16]:
reveneu_by_dept = final_df.groupby('Department')['FinalAmount'].sum()
print(reveneu_by_dept)   # Total revenue per department

Department
Cardiology    4000.0
Neurology     5200.0
Name: FinalAmount, dtype: float64


In [17]:
revenue_by_doc = final_df.groupby('Doctor')['FinalAmount'].sum()
print(revenue_by_doc)    # Total revenue per doctor

Doctor
Dr. John     5200.0
Dr. Smith    4000.0
Name: FinalAmount, dtype: float64


Conclusion:
The hospital patient data was cleaned, merged, and consolidated to create a complete dataset for billing analysis. Missing values were handled, duplicates removed, and new patient records incorporated. The final dataset enables accurate insights into department-wise revenue and doctor performance, ready for further analytics.